In [1]:
import openmesh as om
import networkx as nx
import numpy

def find_middles(t): return (t[0] + t[1])/2, (t[1] + t[2])/2, (t[2] + t[0])/2

def has_vertex(mesh_t,v):
    for i, point in enumerate(mesh_t.points()):
            if all(point == mesh_t.point(v)) and v.idx() != i: return i
    return v.idx()

#triangulates the mesh where every triangle is turned into 6 smaller triangles
def triangulate_mesh (mesh, mesh_t):
    tri_dict = {}
    for fh in mesh.faces():
        triangle = []
        # populate the triangle with points of vertices
        [triangle.append(mesh.point(vertex)) for vertex in mesh.fv(fh)] 
        a,b,c = [mesh_t.add_vertex(point) for point in triangle] 
        d = mesh_t.add_vertex(numpy.mean(triangle,0)) 
        e,f,g = [mesh_t.add_vertex(midpoint) for midpoint in find_middles(triangle)] 
        # check for duplicate vertices, if found, replace with original
        a,b,c,d,e,f,g = [mesh_t.vertex_handle(has_vertex(mesh_t,i)) for i in [a,b,c,d,e,f,g]] 
        #create the six faces--vertices have to be in cw order
        f1 = mesh_t.add_face(e,d,a) 
        f2 = mesh_t.add_face(b,d,e)
        f3 = mesh_t.add_face(f,d,b)
        f4 = mesh_t.add_face(c,d,f)
        f5 = mesh_t.add_face(g,d,c)
        f6 = mesh_t.add_face(a,d,g)
        # remove vertices that used to be duplicates
        mesh_t.delete_isolated_vertices() 
        # clean up the mesh
        mesh_t.garbage_collection()
        #add the faces to the dictionary
        tri_dict[fh.idx()] = [f1.idx(),f2.idx(),f3.idx(),f4.idx(),f5.idx(),f6.idx()]
    return mesh_t, tri_dict

# creates a graph based on mesh data
def graph_from_mesh(m):
    G = nx.Graph()
    # populate the graph with nodes (faces), their idx in the mesh corresponds to the id in the graph
    [G.add_node(face.idx()) for face in m.faces()]
    # add an edge to the graph between the current face and the adjacent face
    [[G.add_edge(node,adj_face.idx()) for adj_face in m.ff(m.face_handle(node))] for node in G.nodes()]
    return G

# returns whether face on the mesh was visited 
def is_visited (mesh,face):
    return mesh.face_property("visited", mesh.face_handle(face))

# returns the next face to continue traversing with
def next_face (adj_faces, faces_on_other_nodes, curr_face, mesh, faces_on_curr_node): 
    candidate = curr_face
    # checks if there is an adjacent face on one of the next_triangles, if so, return that triangle to continue traversing
    for face in adj_faces:
        if (face in [item for sublist in faces_on_other_nodes for item in sublist]):
            candidate = face
            break
    # if we have no candidate yet or the one we found is already visited, we need to look on the current node
    if (candidate == curr_face or is_visited(mesh,face) == True):
        for face in adj_faces:
            # check if face is unvisted and on the current node
            if ( is_visited(mesh,face) != True and face in faces_on_curr_node): 
                candidate = face
                break
    return candidate

# checks if f is on one of the faces in "nodes", if not it must be on the current node 
def face_in_nodes (face,nodes,graph_map, curr_node):    
    for node in nodes:
        if face in graph_map[node]:
            return node    
    return curr_node

# traversal function. l = list of triangles, m = mesh, T = spanning Tree
def traverse(l,m,T, graph_map, node, face):
    # stop when we return to a visited face
    if (is_visited(m,face) == True): return l
    # flag the face as visited
    m.set_face_property("visited", m.face_handle(face), True)
    # add the face to the list 
    l.append(face)
    # find the adjacent nodes in the spanning tree
    adj_nodes = nx.descendants_at_distance(T,node,1)
    # find what faces belong to those nodes
    faces_on_nodes = [graph_map[i] for i in adj_nodes]   
    # find what triangles are adjacent to the current one
    adj_faces = [f.idx() for f in m.ff(m.face_handle(face))]
    # find the next face, either the next on the current node OR an adjacent face on another node
    face = next_face(adj_faces, faces_on_nodes, face, m, graph_map[node])
    # recursively call the traversal method with the next node and triangle
    return traverse(l,m,T, graph_map, face_in_nodes(face,adj_nodes, graph_map, node), face)

# the main algorithm: converts a mesh to a foldable list of triangles
def hamiltonian_refinement(mesh):
    mesh_t = om.TriMesh()
    # triangulate the mesh and populate dictionary 
    mesh_t, graph_map = triangulate_mesh(mesh,mesh_t) 
    # find the spanning tree
    T = nx.minimum_spanning_tree(graph_from_mesh(mesh)) 
    # the list of triangles we will have to fold
    triangle_list = [] 
    #call recursive traversal function
    triangle_list = traverse(triangle_list,mesh_t, T, graph_map,0,0) 
    #return the actual triangles of the list
    return [[(mesh_t.point(vertex)) for vertex in mesh_t.fv(mesh_t.face_handle(t))] for t in triangle_list]
    
mesh = om.TriMesh()
mesh = om.read_trimesh("cube.stl")
print(hamiltonian_refinement(mesh))

[[array([ 5., -2., 10.]), array([5.        , 4.66666667, 6.66666667]), array([ 5., -2., 20.])], [array([ 5., -2., 20.]), array([-1.66666667, -2.        , 13.33333333]), array([ 5., -2., 10.])], [array([-5., -2., 20.]), array([-1.66666667, -2.        , 13.33333333]), array([ 5., -2., 20.])], [array([-15.,  -2.,  20.]), array([-1.66666667, -2.        , 13.33333333]), array([-5., -2., 20.])], [array([-5., -2., 10.]), array([-1.66666667, -2.        , 13.33333333]), array([-15.,  -2.,  20.])], [array([ 5., -2.,  0.]), array([-1.66666667, -2.        , 13.33333333]), array([-5., -2., 10.])], [array([ 5., -2., 10.]), array([-1.66666667, -2.        , 13.33333333]), array([ 5., -2.,  0.])], [array([ 5., -2.,  0.]), array([5.        , 4.66666667, 6.66666667]), array([ 5., -2., 10.])], [array([5., 8., 0.]), array([5.        , 4.66666667, 6.66666667]), array([ 5., -2.,  0.])], [array([ 5., -2.,  0.]), array([-1.66666667, 11.33333333,  0.        ]), array([5., 8., 0.])], [array([-5.,  8.,  0.]), arr